In [1]:
import pandas as pd
from datetime import date
pd.set_option('display.max_columns', None)


In [2]:
df_work_raw = catalog.load('raw/openalex/work_dev#parquet')

                    INFO     Loading data from raw/openalex/work_dev#parquet                    ]8;id=157999;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=602531;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

# Profiling

In [3]:
df_work_raw.shape

(1000, 51)

In [4]:
df_work_raw.head(3)

,id,doi,title,display_name,publication_year,publication_date,ids,language,primary_location,type,type_crossref,indexed_in,open_access,authorships,institution_assertions,countries_distinct_count,institutions_distinct_count,corresponding_author_ids,corresponding_institution_ids,apc_list,apc_paid,fwci,has_fulltext,fulltext_origin,cited_by_count,citation_normalized_percentile,cited_by_percentile_year,biblio,is_retracted,is_paratext,primary_topic,topics,keywords,concepts,mesh,locations_count,locations,best_oa_location,sustainable_development_goals,grants,datasets,versions,referenced_works_count,referenced_works,related_works,abstract_inverted_index,abstract_inverted_index_v3,cited_by_api_url,counts_by_year,updated_date,created_date
0,https://openalex.org/W2089852843,https://doi.org/10.1016/s0378-4371(98)00437-3,The role of constraints within generalized non...,The role of constraints within generalized non...,1998,1998-12-01,{'doi': 'https://doi.org/10.1016/s0378-4371(98...,en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",article,journal-article,[crossref],"{'any_repository_has_fulltext': False, 'is_oa'...",[{'affiliations': [{'institution_ids': array([...,[],3,6,[https://openalex.org/A5051451815],[https://openalex.org/I4210125245],"{'currency': 'USD', 'provenance': 'doaj', 'val...",None,30.137,True,ngrams,1339,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '534', 'issue': '3-4', 'last_pa...",False,False,{'display_name': 'Statistical Mechanics and En...,[{'display_name': 'Statistical Mechanics and E...,[{'display_name': 'Hamiltonian (control theory...,"[{'display_name': 'Harmonic oscillator', 'id':...",[],1,"[{'is_accepted': False, 'is_oa': False, 'is_pu...",None,[{'display_name': 'Affordable and clean energy...,[],[],[],74,"[https://openalex.org/W1553236419, https://ope...","[https://openalex.org/W620348747, https://open...",None,None,https://api.openalex.org/works?filter=cites:W2...,"[{'cited_by_count': 49, 'year': 2024}, {'cited...",2025-01-22T01:09:23.293089,2016-06-24
1,https://openalex.org/W3037375387,https://doi.org/10.1016/j.envint.2020.105876,Short-term exposure to particulate matter (PM1...,Short-term exposure to particulate matter (PM1...,2020,2020-06-23,{'doi': 'https://doi.org/10.1016/j.envint.2020...,en,"{'is_accepted': True, 'is_oa': True, 'is_publi...",review,journal-article,"[crossref, doaj, pubmed]","{'any_repository_has_fulltext': True, 'is_oa':...",[{'affiliations': [{'institution_ids': array([...,[],1,5,[https://openalex.org/A5069680875],"[https://openalex.org/I151201029, https://open...","{'currency': 'USD', 'provenance': 'doaj', 'val...","{'currency': 'USD', 'provenance': 'doaj', 'val...",8.630,True,pdf,525,"{'is_in_top_10_percent': True, 'is_in_top_1_pe...","{'max': 100, 'min': 99}","{'first_page': '105876', 'issue': None, 'last_...",False,False,{'display_name': 'Air Quality and Health Impac...,[{'display_name': 'Air Quality and Health Impa...,"[{'display_name': 'Nitrogen dioxide', 'id': 'h...","[{'display_name': 'Nitrogen dioxide', 'id': 'h...","[{'descriptor_name': 'Air Pollutants', 'descri...",4,"[{'is_accepted': True, 'is_oa': True, 'is_publ...","{'is_accepted': True, 'is_oa': True, 'is_publi...",[{'display_name': 'Good health and well-being'...,"[{'award_id': None, 'funder': 'https://openale...",[],[],85,"[https://openalex.org/W1525116145, https://ope...","[https://openalex.org/W4399338999, https://ope...","{'""Best': None, '""best': None, '#876.': None, ...",None,https://api.openalex.org/works?filter=cites:W3...,"[{'cited_by_count': 170, 'year': 2024}, {'cite...",2025-01-25T03:47:29.633744,2020-07-02
2,https://openalex.org/W2032143374,https://doi.org/10.1016/s0023-6438(03)00088-4,Antimicrobial activity of essential oils on th...,Antimicrobial activity of essential oils on th...,2003,2003-11-01,{'doi': 'https://doi.org/10.1016/s0023-6438(03...,en,"{'is_accepted': False, 'is_oa': False, 'is_pub...",article,journal-article,[crossref],"{'any_repository_ha

In [5]:
df_work_raw = df_work_raw.convert_dtypes()
df_work_raw.dtypes


id                                string[python]
doi                               string[python]
title                             string[python]
display_name                      string[python]
publication_year                           Int64
publication_date                  string[python]
ids                                       object
language                          string[python]
primary_location                          object
type                              string[python]
type_crossref                     string[python]
indexed_in                                object
open_access                               object
authorships                               object
institution_assertions                    object
countries_distinct_count                   Int64
institutions_distinct_count                Int64
corresponding_author_ids                  object
corresponding_institution_ids             object
apc_list                                  object
apc_paid           

# Nodo

In [ ]:
def openalex_land_work_topics(df_work_raw):
    df_work = df_work_raw.loc[:,['id','topics']]
    df_work = df_work.convert_dtypes()
    
    # Proceso topics
    df_work2topics_exploded = df_work.explode('topics')
    df_work2topics_norm = pd.json_normalize(df_work2topics_exploded['topics'])
    df_work2topics_exploded = df_work2topics_exploded.reset_index(drop=True)
    df_work2topics_norm.rename(columns={'id':'topic_id'}, inplace=True)
   
    # Creación de df con work y sus topics
    df_work2topics = pd.concat((df_work2topics_exploded['id'], df_work2topics_norm), axis=1)

    df_work2topics['load_datetime'] = date.today()

    return df_work2topics


## Ejecuto Nodo

In [ ]:
df_work2topics = openalex_land_work_topics(df_work_raw)

# Resultados

In [8]:
df_work2topics

,id,display_name,topic_id,score,domain.display_name,domain.id,field.display_name,field.id,subfield.display_name,subfield.id,load_datetime
0,https://openalex.org/W2089852843,Statistical Mechanics and Entropy,https://openalex.org/T12261,0.9998,Physical Sciences,https://openalex.org/domains/3,Physics and Astronomy,https://openalex.org/fields/31,Statistical and Nonlinear Physics,https://openalex.org/subfields/3109,2025-02-01
1,https://openalex.org/W2089852843,Advanced Thermodynamics and Statistical Mechanics,https://openalex.org/T11520,0.9977,Physical Sciences,https://openalex.org/domains/3,Physics and Astronomy,https://openalex.org/fields/31,Statistical and Nonlinear Physics,https://openalex.org/subfields/3109,2025-02-01
2,https://openalex.org/W2089852843,Fractional Differential Equations Solutions,https://openalex.org/T10288,0.9875,Physical Sciences,https://openalex.org/domains/3,Mathematics,https://openalex.org/fields/26,Modeling and Simulation,https://openalex.org/subfields/2611,2025-02-01
3,https://openalex.org/W3037375387,Air Quality and Health Impacts,https://openalex.org/T10190,1.0000,Physical Sciences,https://openalex.org/domains/3,Environmental Science,https://openalex.org/fields/23,"Health, Toxicology and Mutagenesis",https://openalex.org/subfields/2307,2025-02-01
4,https://openalex.org/W3037375387,Climate Change and Health Impacts,https://openalex.org/T11244,0.9976,Physical Sciences,https://openalex.org/domains/3,Environmental Science,https://openalex.org/fields/23,"Health, Toxicology and Mutagenesis",https://openalex.org/subfields/2307,2025-02-01
...,...,...,...,...,...,...,...,...,...,...,...
2918,https://openalex.org/W2314900886,Advanced ceramic materials synthesis,https://openalex.org/T10132,0.9983,Physical Sciences,https://openalex.org/domains/3,Materials Science,https://openalex.org/fields/25,Ceramics and Composites,https://openalex.org/subfields/2503,2025-02-01
2919,https://openalex.org/W2314900886,Advanced materials and composites,https://openalex.org/T12099,0.9980,Physical Sciences,https://openalex.org/domains/3,Engineering,https://openalex.org/fields/22,Mechanical Engineering,https://openalex.org/subfields/2210,2025-02-01
2920,https://openalex.org/W2738753887,Multisensory perception and integration,https://openalex.org/T12032,0.9992,Social Sciences,https://openalex.org/domains/2,Psychology,https://openalex.org/fields/32,Experimental and Cognitive Psychology,https://openalex.org/subfields/3205,2025-02-01
2921,https://openalex.org/W2738753887,Visual perception and processing mechanisms,https://openalex.org/T10427,0.9968,Life Sciences,https://openalex.org/domains/1,Neuroscience,https://openalex.org/fields/28,Cognitive Neuroscience,https://openalex.org/subfields/2805,2025-02-01
